Convert a list of image-image Relations to absolute relative image positions

In [27]:
from os.path import abspath
from sys import path
path.insert(0, abspath("../.."))  # Repository directory relative to this file.
from MISalign.model.relation import Relation
from MISalign.model.image import Image
from MISalign.model.mis_file import MisFile,save_mis,load_mis

import numpy as np
from matplotlib import pyplot as plt

In [28]:
mf=load_mis(r"..\..\example\data\set_a\mymis.mis")
print(mf.get_rels())

[('a_myimages01.jpg', 'a_myimages02.jpg'), ('a_myimages02.jpg', 'a_myimages03.jpg'), ('a_myimages03.jpg', 'a_myimages04.jpg'), ('a_myimages04.jpg', 'a_myimages05.jpg'), ('a_myimages05.jpg', 'a_myimages06.jpg'), ('a_myimages06.jpg', 'a_myimages07.jpg'), ('a_myimages07.jpg', 'a_myimages08.jpg'), ('a_myimages08.jpg', 'a_myimages09.jpg'), ('a_myimages09.jpg', 'a_myimages10.jpg')]


## Rectangular solve

In [29]:
relations=mf.get_rels('r')
display(relations)
#TODO set/update relations
origin="a_myimages01.jpg"


[[('a_myimages01.jpg', 'a_myimages02.jpg'), (12, -1088)],
 [('a_myimages02.jpg', 'a_myimages03.jpg'), (8, -1141)],
 [('a_myimages03.jpg', 'a_myimages04.jpg'), (-129, -1058)],
 [('a_myimages04.jpg', 'a_myimages05.jpg'), (11, -943)],
 [('a_myimages05.jpg', 'a_myimages06.jpg'), (5, -860)],
 [('a_myimages06.jpg', 'a_myimages07.jpg'), (10, -990)],
 [('a_myimages07.jpg', 'a_myimages08.jpg'), (9, -1009)],
 [('a_myimages08.jpg', 'a_myimages09.jpg'), (9, -939)],
 [('a_myimages09.jpg', 'a_myimages10.jpg'), (3, -722)]]

In [30]:
[origin in r[0] for r in relations]

[True, False, False, False, False, False, False, False, False]

In [31]:
image_names=list(set([name for r in relations for name in r[0]])) #TODO method for getting all the image names
print(image_names)

['a_myimages08.jpg', 'a_myimages03.jpg', 'a_myimages06.jpg', 'a_myimages07.jpg', 'a_myimages04.jpg', 'a_myimages01.jpg', 'a_myimages05.jpg', 'a_myimages09.jpg', 'a_myimages10.jpg', 'a_myimages02.jpg']


In [32]:
# Develop relation map
found=[image_names.index(origin)]
matched=[]
resolved=[]
relation_map=dict({x:[] for x in image_names})

while len(resolved)<len(relations):
    for i in found:
        for ii,x in enumerate(image_names):
            if (ii not in found) & (ii not in resolved) & (ii not in matched):
                i_match=[image_names[i] in r[0] for r in relations]
                ii_match=[image_names[ii] in r[0] for r in relations]
                full_match=[im&iim for im,iim in zip(i_match,ii_match)]
                if any(full_match):
                    relation_map[image_names[i]].append((image_names[ii],relations[full_match.index(True)]))
                    matched.append(ii)
        resolved.append(i)
    found=matched
    matched=[]
    #break if stuck
    if found==[]:
        break

for k,v in relation_map.items():
    print(k)
    for x in v:
        print("   ",x)

a_myimages08.jpg
    ('a_myimages09.jpg', [('a_myimages08.jpg', 'a_myimages09.jpg'), (9, -939)])
a_myimages03.jpg
    ('a_myimages04.jpg', [('a_myimages03.jpg', 'a_myimages04.jpg'), (-129, -1058)])
a_myimages06.jpg
    ('a_myimages07.jpg', [('a_myimages06.jpg', 'a_myimages07.jpg'), (10, -990)])
a_myimages07.jpg
    ('a_myimages08.jpg', [('a_myimages07.jpg', 'a_myimages08.jpg'), (9, -1009)])
a_myimages04.jpg
    ('a_myimages05.jpg', [('a_myimages04.jpg', 'a_myimages05.jpg'), (11, -943)])
a_myimages01.jpg
    ('a_myimages02.jpg', [('a_myimages01.jpg', 'a_myimages02.jpg'), (12, -1088)])
a_myimages05.jpg
    ('a_myimages06.jpg', [('a_myimages05.jpg', 'a_myimages06.jpg'), (5, -860)])
a_myimages09.jpg
    ('a_myimages10.jpg', [('a_myimages09.jpg', 'a_myimages10.jpg'), (3, -722)])
a_myimages10.jpg
a_myimages02.jpg
    ('a_myimages03.jpg', [('a_myimages02.jpg', 'a_myimages03.jpg'), (8, -1141)])


In [33]:
absolute_position={origin:(0,0)}
solving=[origin]
cansolve=[]
solved=[]
while len(solving)>0:
    for s in solving:
        for image_name,rel in relation_map[s]:
            cansolve.append(image_name)
            if rel[0][0]==s:
                direction=1
            else:
                direction=-1
            absolute_position[image_name]=(absolute_position[s][0]+direction*rel[1][0],absolute_position[s][1]+direction*rel[1][1])
    solved+=solving
    solving=cansolve
    cansolve=[]
display(absolute_position)

{'a_myimages01.jpg': (0, 0),
 'a_myimages02.jpg': (12, -1088),
 'a_myimages03.jpg': (20, -2229),
 'a_myimages04.jpg': (-109, -3287),
 'a_myimages05.jpg': (-98, -4230),
 'a_myimages06.jpg': (-93, -5090),
 'a_myimages07.jpg': (-83, -6080),
 'a_myimages08.jpg': (-74, -7089),
 'a_myimages09.jpg': (-65, -8028),
 'a_myimages10.jpg': (-62, -8750)}

In [ ]:
#TODO maxima/minima adjustment for non-purely positive sets
    #TODO Images sizes.